In [1]:
from data import data
from analyze import analyze

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
def mmm(x=3,age=2017, ind="GDP per capita (current US$)"):
    df1 = analyze(data, age, [ind])
    df1 = df1.sort_values([ind])
    lmd1 = int(len(df1)/2)
    return df1.iloc[list(range(0,x))+list(range(lmd1-int(x/2), lmd1+1+int(x/2)))+list(range(-x,0))]

In [ ]:
mmm(x=10,ind="Total tax rate (% of commercial profits)").plot(kind="bar")
plt.show()

In [ ]:
analyze(data,2017)

In [ ]:
a = pd.read_excel('https://www.heritage.org/index/excel/2013/index2013_data.xls')

In [ ]:
import urllib2
response = urllib2.urlopen('https://www.heritage.org/index/excel/2013/index2013_data.xls')